In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import panel as pn
import time 
import threading
import queue

import os
import glob
from joblib import Parallel, delayed

from collections import OrderedDict
import src.doc_processing as processing 

pn.extension()
pn.extension('texteditor')

In [4]:
# from src.twopanes import TwoPanes
# from src.twopanes  import SimpleTwoPanes
# from src.llm_handler import LLMHandler

In [3]:
def split_text_into_chunks(text, chunk_size=5):
    """Split the input text into fixed-size chunks."""
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def display_text(rightpane, text, delay=0.005):
    # Reset the current state
    
    index = 0
    text_chunks = split_text_into_chunks(text)
    # Using a while loop to print each chunk with a delay
    while index < len(text_chunks):
        new_content = rightpane.object + f"{text_chunks[index]}"
        rightpane.object = new_content         
        index += 1
        time.sleep(delay)
        
   

In [4]:
import panel as pn
pn.extension()
pn.extension('texteditor')
import panel as pn
import glob 
from collections import OrderedDict
import src.doc_processing as processing 
import os 
from src.panetexteditor import PaneTextEditorToggle

class TwoPanes:
    def __init__(self, location, queue):
        self.location = location
        self.q = queue  
        
        # Define widgets and button action
        self.dropdown1 = pn.widgets.Select(name='Document', options=[os.path.splitext(file)[0] for file in self.get_filenames(['html', 'pdf' ])])
        self.dropdown2 = pn.widgets.Select(name='Prompt', options=['Summarize with images', 'Summarize concise', 'Summarize detailed'])
        self.run_button = pn.widgets.Button(name='Summarize', button_type='primary')
        self.clear_button = pn.widgets.Button(name='Clear', button_type='primary')

        self.run_button.on_click(self.on_run_click)
        self.clear_button.on_click(self.on_clear_click)

        # Update whenever the dropdown value changes
        self.dropdown1.param.watch(lambda event: self.update_left_pane_content(), 'value')

        self.left_content = "<p>Left Pane Content...</p>"
        self.right_content = "<p>Summary Place Holder...</p>"
        # Content panes

        self.left_pane_content = pn.panel(self.left_content, 
                                          sizing_mode='stretch_both', 
                                          styles={'border-radius': '8px', 'overflow': 'auto'}, 
                                          min_height=150,
                                          margin=(5,5))

 
        
        self.right_pane_content = PaneTextEditorToggle(self.right_content)
        
        self.update_left_pane_content()

    def get_filenames(self, extensions): 
        files = [file for ext in extensions for file in glob.glob(f"{self.location}/*.{ext}")]
        filenames  = OrderedDict()
        for file in files:
            filenames[os.path.basename(file)] = file
        return filenames  

    def update_left_pane_content(self):
        print("Update Left pane Content")
 
        file_path = os.path.join(f'{self.location}/{self.dropdown1.value}.html')
        with open(file_path, 'r', encoding='utf-8') as file:
            self.left_content = file.read()

        self.left_pane_content.object = self.left_content


    def _get_processing_content(self):
        print("get content")
        return self.left_content

    def _get_right_content(self):
        return self.right_content        
 

    def on_run_click(self, event):   
        print("Clicked Run")
        self.right_pane_content.object = "<p><strong>Summarizing...</strong></p>"

        self.doc_parsed = processing.parse_html(self._get_processing_content()) 
        requests = processing.process_content_request(self.doc_parsed)
        processed_requests = Parallel(n_jobs=16)(delayed(processing.text_processing2)(request) for request in requests)
        sorted_processed_requests = sorted(processed_requests, key=lambda x: x.request_id)
        self.right_pane_content.object = "<div></div>"
        for req in sorted_processed_requests: 
            display_text(self.right_pane_content, req.header, delay=0.005)
            display_text(self.right_pane_content, req.content, delay=0.005) 
        self.right_content = self.right_pane_content.object
        print("Done summarizing")
 
    def on_clear_click(self, event):
        self.right_pane_content.object = "<p>Summary Place Holder...</p>"
        
    def get_pane_layout(self):
        # Logic to get the final pane layout 
        run_button_col = pn.Column(pn.Spacer(height=10), 
                                   pn.Row(self.run_button, self.clear_button) , align="end", 
                                   sizing_mode='stretch_width')

        top_row = pn.Row(self.dropdown1, self.dropdown2, run_button_col,  align="end", sizing_mode='stretch_width')

        left_column = pn.Column(top_row, pn.layout.Divider(), self.left_pane_content, sizing_mode='stretch_both', margin=(5,5))
        right_column = pn.Column(self.right_pane_content, sizing_mode='stretch_both', margin=(5,5))

        tab1 = pn.GridSpec(sizing_mode='stretch_both', ncols=12, nrows=1,)
        tab1[0, :6] = left_column
        tab1[0, 6:] = right_column

        return tab1

    def view(self):
        return self.get_pane_layout()

location = "C:/Users/enisb/python/gen-ai/documents/docx-extraction"

q = queue.Queue() 
two_pane = TwoPanes(location, q)
two_pane.view()

Update Left pane Content


Clicked Run 
get content 
Done summarizing 
Update Left pane Content

GridSpec(ncols=12, nrows=1, sizing_mode='stretch_both')
    [0] Column(margin=(5, 5), sizing_mode='stretch_both')
        [0] Row(align='end', sizing_mode='stretch_width')
            [0] Select(name='Document', options=['01_executive-summary', ...], value='01_executive-summary')
            [1] Select(name='Prompt', options=['Summarize with images', ...], value='Summarize with images')
            [2] Column(align='end', sizing_mode='stretch_width')
                [0] Spacer(height=10)
                [1] Row
                    [0] Button(button_type='primary', name='Summarize')
                    [1] Button(button_type='primary', name='Clear')
        [1] Divider()
        [2] Markdown(str, margin=(5, 5), min_height=150, sizing_mode='stretch_both', styles={'border-radius': '8px', ...})
    [1] Column(margin=(5, 5), sizing_mode='stretch_both')
        [0] PaneTextEditorToggle
            [0] Row(align='end')
                [0] Button(name='Edit', width=100)
            [1] Markdown(str, margin=(5, 5), min_height=150, styles={'border-radius': '8px', ...})

In [5]:
location = "C:/Users/enisb/python/gen-ai/documents/docx-extraction"

q = queue.Queue() 
two_pane = TwoPanes(location, q)
printProcessor = PrintProcessor(q, two_pane.right_pane_content)
printProcessor.start()
 
model = LLMHandler(project_id = "llm-project-401322", location = "us-central1", credentials_path = "../llm-sa2.json")
pane = SimpleTwoPanes(model = model)

Update Left pane Content


In [6]:
new_tab_content = pn.pane.Markdown('This tab is currently placehoder.')

tabs = pn.Tabs(('Summarization', two_pane.view()), ('Model Prompt', pane.view()))

# Sidebar placeholder (replace with your sidebar content)
sidebar = pn.pane.Markdown('### Sidebar Content')

# Define the template and append tabs
template = pn.template.BootstrapTemplate(title='Model Comparison Dashboard', sidebar=sidebar)
template.main.append(tabs)

template.show()


Launching server at http://localhost:53241


Clicked Run
Done summarizing
Clicked Run
Done summarizing
Clicked Run
Done summarizing


In [10]:
import panel as pn

# Your markdown content with embedded CSS to adjust font size
markdown_content = """
<style>
    .bk {
        font-size: 30px; /* adjust as needed */
    }
</style>

# This is a heading

This is some markdown content with a larger font size.
"""

# Create a panel with the markdown content
markdown_panel = pn.panel(markdown_content, styles = {'font-size': '20px'})

# Display the panel
markdown_panel 


Markdown(str, styles={'font-size': '20px'})